In [1]:
import pandas as pd
import os
import numpy as np
import proplot as plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import geopandas as gpd
import xarray as xr
import rioxarray as rioxr
os.getcwd()

'D:\\Py_works\\00Dh_process'

In [2]:
pth_nc='D:\\Key_works\\20231212dh_calculate\\xnhl_test\\gla_level_dh_final'

In [3]:
pth='D:/Key_works/20231212dh_calculate/xnhl_test'
df_log=pd.read_csv(pth+'/dh-effe-stat.csv')
df_log.head(1)

,Unnamed: 0,group_id,mean,median,mean_fill,median_fill,Glc_Area,Mean_Elev,Mean_Slp,Mean_Asp,MA_Elev,basin
0,G101096E33298N,g0018,-2.247736,-4.501035,-2.532606,-4.124494,293031.2539,4942.3,39.4,315.8,4939.9,Yellow


In [4]:
stat_df=list(df_log.iloc[:,0])

In [11]:
my_prj='''PROJCS["Albers_Conic_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",105],PARAMETER["standard_parallel_1",25],PARAMETER["standard_parallel_2",47],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'''

In [24]:
newp0=pth_nc+'\\nc_yr'
newp1=pth_nc+'\\nc_yr_mb'
if os.path.exists(newp1):pass
else:os.makedirs(newp1)

aaa=[]
for glai in stat_df: #G075167E36993N stat_df
    innm=newp0+'/'+glai+'.nc'
    outnm=newp1+'/'+glai+'.nc'
    ds_gla=xr.open_dataset(innm)

    ds_gla['mb']=(ds_gla['topo_dh']/(2000-ds_gla['dxt_id'])*0.85)
    t=(2000-ds_gla['dxt_id']).mean()
    pi=0.85;pw=1;dp=0.06
    mb_err=(((ds_gla['topo_dh'].mean()/t)*(dp/pw))**2+((ds_gla.attrs['dh error']/t)*(pi/pw))**2)**0.5
    ds_gla['mb'].attrs={'long_name' :'Mass balance (by using [topo_dh])',
                        'units' :'m w.e. yr-1'}
    ds_gla['dxt_id'].attrs={'long_name' :'Photography time of aerial photos (Topo maps time)',}
    ds_gla.attrs['mb error']=round(float(mb_err.values),4)
    aaa.append([ds_gla['mb'].mean(),mb_err.values])
    ds_gla0=ds_gla.rio.write_crs(my_prj)
    encd={'topo_dh':{"zlib": True, "complevel": 8},'topo_dh_fill':{"zlib": True, "complevel": 8},
         'elevation':{"zlib": True, "complevel": 8},'dxt_id':{"zlib": True, "complevel": 8},
         'mb':{"zlib": True, "complevel": 8},}
    ds_gla0.to_netcdf(outnm)
    #break

### 重投影

In [5]:
newp1=pth_nc+'\\nc_yr_mb'
newp2=pth_nc+'\\Glacier-level mass change'

In [6]:
flst=[i for i in os.listdir(newp1) if i[-3:]=='.nc']

In [17]:
for glai in flst: #G075167E36993N stat_df
    innm=newp1+'/'+glai
    outnm=newp2+'/'+glai
    ds_gla=xr.open_dataset(innm)
    ds_gla0=ds_gla.rio.write_crs(my_prj)
    ds_gla1=ds_gla0.rio.reproject('epsg:4326')
    encd={'topo_dh':{"zlib": True, "complevel": 8},'topo_dh_fill':{"zlib": True, "complevel": 8},
         'elevation':{"zlib": True, "complevel": 8},'dxt_id':{"zlib": True, "complevel": 8},
         'mb':{"zlib": True, "complevel": 8},}
    ds_gla1.to_netcdf(outnm)
    #aaa

In [15]:
ds_gla1

<xarray.Dataset> Size: 74kB
Dimensions:       (x: 56, y: 41)
Coordinates:
  * x             (x) float64 448B 78.74 78.74 78.74 78.74 ... 78.76 78.76 78.76
  * y             (y) float64 328B 31.19 31.19 31.19 31.19 ... 31.18 31.18 31.18
    spatial_ref   int32 4B 0
Data variables:
    topo_dh       (y, x) float64 18kB nan nan nan nan nan ... nan nan nan nan
    elevation     (y, x) float32 9kB nan nan nan nan nan ... nan nan nan nan nan
    topo_dh_fill  (y, x) float64 18kB nan nan nan nan nan ... nan nan nan nan
    dxt_id        (y, x) float32 9kB nan nan nan nan nan ... nan nan nan nan nan
    mb            (y, x) float64 18kB nan nan nan nan nan ... nan nan nan nan
Attributes:
    transform:           [ 3.00000000e+01  0.00000000e+00 -2.43928035e+06  0....
    res:                 [30. 30.]
    nodatavals:          nan
    scales:              1.0
    offsets:             0.0
    topo process group:  g0309
    effective samples:   17.4075
    dh error:            2.5313
    mb error:            0.2098

In [14]:
ds_gla0

<xarray.Dataset> Size: 15kB
Dimensions:       (x: 22, y: 21)
Coordinates:
  * x             (x) float64 176B -3.58e+05 -3.58e+05 ... -3.574e+05 -3.574e+05
  * y             (y) float64 168B 3.55e+06 3.55e+06 ... 3.549e+06 3.549e+06
    spatial_ref   int32 4B 0
Data variables:
    topo_dh       (y, x) float64 4kB nan nan nan nan nan ... nan nan nan nan nan
    elevation     (y, x) float32 2kB ...
    topo_dh_fill  (y, x) float64 4kB ...
    dxt_id        (y, x) float32 2kB nan nan nan nan nan ... nan nan nan nan nan
    mb            (y, x) float64 4kB nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    transform:           [ 3.00000000e+01  0.00000000e+00 -3.58000351e+05  0....
    res:                 [30. 30.]
    nodatavals:          nan
    scales:              1.0
    offsets:             0.0
    topo process group:  g0018
    effective samples:   10.5176
    dh error:            3.6774
    mb error:            0.0948

In [42]:
dff_tst=pd.DataFrame(np.array(aaa),columns=['mb','mb_err'])

In [47]:
dff_tst['mb'].mean(),dff_tst['mb_err'].median()

(-0.31040319417826173, 0.1228385022605827)

In [28]:
mb_err.values

array(0.09480903)

In [37]:
ds_gla

<xarray.Dataset> Size: 15kB
Dimensions:       (x: 22, y: 21)
Coordinates:
  * x             (x) float64 176B -3.58e+05 -3.58e+05 ... -3.574e+05 -3.574e+05
  * y             (y) float64 168B 3.55e+06 3.55e+06 ... 3.549e+06 3.549e+06
Data variables:
    topo_dh       (y, x) float64 4kB nan nan nan nan nan ... nan nan nan nan nan
    elevation     (y, x) float32 2kB ...
    topo_dh_fill  (y, x) float64 4kB ...
    dxt_id        (y, x) float32 2kB nan nan nan nan nan ... nan nan nan nan nan
    mb            (y, x) float64 4kB nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    transform:           [ 3.00000000e+01  0.00000000e+00 -3.58000351e+05  0....
    crs:                 +proj=aea +lat_0=0 +lon_0=105 +lat_1=25 +lat_2=47 +x...
    res:                 [30. 30.]
    nodatavals:          nan
    scales:              1.0
    offsets:             0.0
    topo process group:  g0018
    effective samples:   10.5176
    dh error:            3.6774
    mb error:            0.0948